In [1]:
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning
from data_processing import DataProcessor
from IPython.display import display
import pandas as pd
import tabula
import ssl

In [2]:
creds_path = '/Users/itsanya/AiCore/MRDC/db_creds.yaml'
db_connector_instance = DatabaseConnector(creds_path) 
data_extractor = DataExtractor(db_connector_instance) 
data_cleaner = DataCleaning()
data_processor = DataProcessor() 

In [3]:
#Legacy Users
#Extract
#table_name = 'legacy_users'
#users_df = data_extractor.read_rds_table(table_name)

#View
#display(users_df)

#Clean
#fully_cleaned_users_df = data_cleaner.clean_users_df(users_df)

#View Cleaned DataFrame
#display(fully_cleaned_users_df)

#Upload to dim_users
#db_connector_instance.upload_to_db(df=fully_cleaned_users_df, table_name='dim_users')

In [6]:
#Card Data
#Extract
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
card_df = data_extractor.retrieve_pdf_data(pdf_link)

#View
#display(card_df)

#column: card_details
#shortest_card_number = min(card_df['card_number'], key=len)
#print("Value with the shortest length:", shortest_card_number)

#column: card_provider
#unique_prov = card_df['card_provider'].unique()
#print(unique_prov)

#Clean
#fully_cleaned_card_df = data_cleaner.clean_card_data(card_df)

#View Cleaned DataFrame
#display(fully_cleaned_card_df)

#Upload to dim_users
#db_connector_instance.upload_to_db(df=fully_cleaned_card_df, table_name='dim_card_details')

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>

In [ ]:
#Legacy Store Details
#Extract
#table_name = 'legacy_store_details'
#store_df = data_extractor.read_rds_table(table_name)

#Clean


#View Cleaned Data
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(clean_store_df)

In [ ]:
#Orders Table
#Extract
#table_name = 'orders_table'
#orders_df = data_extractor.read_rds_table(table_name)

#Clean


#View Cleaned Data
#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    #display(clean_store_df)

#Upload to ... 